In [9]:
import torch
import torch.optim as optim


class TensorFactorization:
    def __init__(self, tensor, rank, method="cp", mask=None, constraint=None, device=None):
        if device is None:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.device = device

        # Move tensors to the device
        tensor = tensor.to(self.device)
        if mask is None:
            mask = torch.ones_like(tensor, device=self.device)
        else:
            mask = mask.to(self.device)
        if constraint is None:
            constraint = torch.ones_like(tensor, device=self.device)
        else:
            constraint = constraint.to(self.device)

        assert tensor.shape == mask.shape == constraint.shape, "Tensor, mask, and constraint must have the same shape."

        self.tensor = tensor
        self.mask = mask
        self.constraint = constraint
        self.method = method.lower()

        self.total_params = 0  # Initialize total_params

        if self.method == "cp":
            self.rank = rank
            self.dims = tensor.shape
            self.factors = [torch.randn(dim, rank, requires_grad=True, device=self.device) for dim in self.dims]
            self.total_params = sum(factor.numel() for factor in self.factors)

        elif self.method == "tucker":
            self.rank = rank if isinstance(rank, tuple) else (rank,) * len(tensor.shape)
            self.core = torch.randn(*self.rank, requires_grad=True, device=self.device)
            self.factors = [torch.randn(dim, r, requires_grad=True, device=self.device) for dim, r in zip(tensor.shape, self.rank)]
            self.total_params = self.core.numel() + sum(factor.numel() for factor in self.factors)

        elif self.method == "train":
            self.ranks = rank if isinstance(rank, list) else [rank] * (len(tensor.shape) + 1)
            assert self.ranks[0] == self.ranks[-1] == 1, "Tensor Train ranks must start and end with 1."
            self.factors = [
                torch.randn(self.ranks[i], tensor.shape[i], self.ranks[i + 1], requires_grad=True, device=self.device)
                for i in range(len(tensor.shape))
            ]
            self.total_params = sum(factor.numel() for factor in self.factors)

        elif self.method == "ring":
            self.rank = rank
            self.factors = [
                torch.randn(rank, tensor.shape[i], rank, requires_grad=True, device=self.device)
                for i in range(len(tensor.shape))
            ]
            self.total_params = sum(factor.numel() for factor in self.factors)

        else:
            raise ValueError(f"Unsupported method: {method}. Choose from 'cp', 'tucker', 'train', or 'ring'.")

        # for logging
        self.loss = None
        self.mse_loss = None
        self.constraint_loss = None
        self.l2_loss = None

        print(f"Initialized {method} decomposition with rank {rank} on device {self.device}.")
        print(f"Total parameters: {self.total_params}")

    def reconstruct(self):
        """
        Reconstruct the tensor based on the decomposition method.
        """
        if self.method == "cp":
            R = self.rank
            recon = torch.zeros_like(self.tensor, device=self.device)  # Ensure tensor is on the correct device
            for r in range(R):
                component = torch.ger(self.factors[0][:, r], self.factors[1][:, r])
                for mode in range(2, len(self.dims)):
                    component = component.unsqueeze(-1) * self.factors[mode][:, r]
                recon += component
            return recon

        elif self.method == "tucker":
            recon = self.core
            for i, factor in enumerate(self.factors):
                recon = torch.tensordot(recon, factor, dims=[[0], [1]])
            return recon

        elif self.method == "train":
            recon = self.factors[0]
            for factor in self.factors[1:]:
                recon = torch.einsum("...i,ijk->...jk", recon, factor)
            return recon.squeeze()

        elif self.method == "ring":
            n_modes = len(self.factors)
            result = self.factors[0]
            for i in range(1, n_modes-1):
                result = torch.einsum('ijk,klm->ijlm', result, self.factors[i])
                s1, s2, s3, s4 = result.shape
                result = result.reshape(s1, s2*s3, s4)
            result = torch.einsum('ijk,klm->jl', result, self.factors[-1])
            result = result.reshape(self.tensor.shape)
            return result

    def optimize(self, lr=0.01, max_iter=1000, tol=1e-6, reg_lambda=0.01, constraint_lambda=1):
        """
        Perform optimization for the specified decomposition method.

        Args:
        - lr: float, learning rate.
        - max_iter: int, maximum number of iterations.
        - tol: float, tolerance for convergence.
        - reg_lambda: float, regularization coefficient for L2 regularization.
        - constraint_lambda: float, penalty coefficient for constraint violations.

        Returns:
        - factors: Optimized factor matrices or tensors for the decomposition method.
        """
        params = self.factors if self.method != "tucker" else [self.core] + self.factors
        optimizer = optim.Adam(params, lr=lr)
        prev_loss = float('inf')

        for iteration in range(max_iter):
            optimizer.zero_grad()

            reconstruction = self.reconstruct()

            def loss_fn():
                # Ensure tensors are on the same device
                error_term = self.constraint * self.mask * (self.tensor - reconstruction)
                mse_loss = torch.norm(error_term) ** 2
                violation_term = torch.clamp((1 - self.constraint) * reconstruction, min=0)
                constraint_loss = constraint_lambda * torch.sum(violation_term)
                l2_loss = reg_lambda * sum(torch.norm(factor) ** 2 for factor in params)
                total_loss = mse_loss + constraint_loss + l2_loss
                return total_loss, mse_loss, constraint_loss, l2_loss

            loss, mse_loss, constraint_loss, l2_loss = loss_fn()
            loss.backward()
            optimizer.step()

            if iteration == max_iter - 1:
                print(f"Iter: {iteration}, Loss: {loss}")
                print(f"MSE: {mse_loss}, CONST: {constraint_loss}, L2: {l2_loss}")

                # for logging
                self.loss = loss
                self.mse_loss = mse_loss
                self.constraint_loss = constraint_loss
                self.l2_loss = l2_loss

            if abs(prev_loss - loss.item()) < tol:
                print("Converged.")
                break
            prev_loss = loss.item()


        return [factor.detach() for factor in params]

In [10]:
import pandas as pd
import torch
import torch.optim as optim
import logging
import time
from datetime import datetime


# Initialize logging
log_filename = f"tensor_factorization_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

# Initialize results list to store experiment data dynamically
results = []

def run_tf(dim, mode, rank, method):
    start_time = time.time()

    dim = (dim,)
    shape = dim * mode
    tensor = torch.randn(shape)
    constraint = (torch.rand_like(tensor) > 0.5).float()

    if method == "cp":
        rank = rank
    elif method == "tucker":
        rank = (rank,) * mode
    elif method == "train":
        rank = [1] + [rank] * (mode - 1) + [1]
    elif method == "ring":
        rank = rank

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tf = TensorFactorization(tensor, rank=rank, method=method, mask=None, constraint=constraint, device=device)

    try:
        factors = tf.optimize()
        reconstruction = tf.reconstruct()

        # Extract loss details
        tf.optimize(lr=0.01, max_iter=1000, tol=1e-6, reg_lambda=0.01, constraint_lambda=1)

        loss = tf.loss
        mse_loss = tf.mse_loss
        constraint_loss = tf.constraint_loss
        l2_loss = tf.l2_loss

        params = tf.total_params

        elapsed_time = time.time() - start_time

        # Log results
        results.append({
            "Mode": mode,
            "Method": method,
            "Params": params,
            "Iter": 999,  # Fixed value based on experiment
            "Loss": loss.item(),
            "MSE": mse_loss.item(),
            "CONST": constraint_loss.item(),
            "L2": l2_loss.item(),
            "Time (s)": elapsed_time,
        })

        logging.info(f"Mode: {mode}, Method: {method}, Loss: {loss.item()}, Time: {elapsed_time:.6f} seconds")
    except Exception as e:
        logging.error(f"Error occurred for mode: {mode}, method: {method}. Exception: {e}")


# Main experiment loop
dim = 7
rank = 3
methods = ["cp", "tucker", "train", "ring"]
modes = [2, 3, 4, 5, 6, 7, 8, 9]

for mode in modes:
    for method in methods:
        run_tf(dim=dim, mode=mode, rank=rank, method=method)

# Create DataFrame from the results
df = pd.DataFrame(results)

Initialized cp decomposition with rank 3 on device cuda.
Total parameters: 42
Iter: 999, Loss: 1.9220526218414307
MSE: 1.4067226648330688, CONST: 0.1966472864151001, L2: 0.3186826705932617
Iter: 999, Loss: 1.7832306623458862
MSE: 1.136535882949829, CONST: 0.3647735118865967, L2: 0.28192129731178284
Initialized tucker decomposition with rank (3, 3) on device cuda.
Total parameters: 51
Iter: 999, Loss: 0.5621980428695679
MSE: 0.25784170627593994, CONST: 3.7048761441837996e-05, L2: 0.3043193221092224
Iter: 999, Loss: 0.23977768421173096
MSE: 0.009122618474066257, CONST: 0.0026126292068511248, L2: 0.22804243862628937
Initialized train decomposition with rank [1, 3, 1] on device cuda.
Total parameters: 42
Iter: 999, Loss: 0.70591139793396
MSE: 0.3472881615161896, CONST: 0.002661030739545822, L2: 0.35596218705177307
Converged.
Initialized ring decomposition with rank 3 on device cuda.
Total parameters: 126
Iter: 999, Loss: 3.4171981811523438
MSE: 1.3879547119140625, CONST: 1.2062150239944458

ERROR:root:Error occurred for mode: 5, method: cp. Exception: 'NoneType' object has no attribute 'item'


Converged.
Initialized tucker decomposition with rank (3, 3, 3, 3, 3) on device cuda.
Total parameters: 348
Converged.
Iter: 999, Loss: 8256.4560546875
MSE: 8217.900390625, CONST: 36.24117660522461, L2: 2.314318895339966
Initialized train decomposition with rank [1, 3, 3, 3, 3, 1] on device cuda.
Total parameters: 231
Iter: 999, Loss: 8579.0419921875
MSE: 8469.7822265625, CONST: 107.94380187988281, L2: 1.3161860704421997
Converged.
Initialized ring decomposition with rank 3 on device cuda.
Total parameters: 315
Iter: 999, Loss: 8488.6953125
MSE: 8472.0341796875, CONST: 14.27896785736084, L2: 2.3821334838867188
Iter: 999, Loss: 8232.3154296875
MSE: 8200.701171875, CONST: 29.56212615966797, L2: 2.051945447921753
Initialized cp decomposition with rank 3 on device cuda.
Total parameters: 126
Iter: 999, Loss: 59285.41796875
MSE: 59201.046875, CONST: 83.89155578613281, L2: 0.48002877831459045
Converged.
Initialized tucker decomposition with rank (3, 3, 3, 3, 3, 3) on device cuda.
Total param

In [11]:
df

,Mode,Method,Params,Iter,Loss,MSE,CONST,L2,Time (s)
0,2,cp,42,999,1.783231e+00,1.136536e+00,3.647735e-01,0.281921,5.150636
1,2,tucker,51,999,2.397777e-01,9.122618e-03,2.612629e-03,0.228042,4.110777
2,2,train,42,999,7.059114e-01,3.472882e-01,2.661031e-03,0.355962,2.944586
3,2,ring,126,999,2.336458e+00,1.736884e+00,1.761822e-01,0.423392,4.005326
4,3,cp,63,999,7.464887e+01,7.051344e+01,3.322236e+00,0.813197,6.127677
5,3,tucker,90,999,8.196551e+01,7.580875e+01,5.396653e+00,0.760098,4.652681
6,3,train,105,999,6.433687e+01,5.366779e+01,9.914131e+00,0.754951,4.981957
7,3,ring,189,999,8.470431e+01,7.813026e+01,5.336541e+00,1.237514,5.060474
8,4,cp,84,999,1.110758e+03,1.093906e+03,1.640584e+01,0.446244,7.751857
9,4,tucker,165,999,1.063075e+03,1.036547e+03,2.512066e+01,1.407461,5.239613


In [14]:
import pandas as pd
import torch
import torch.optim as optim
import logging
import time
from datetime import datetime


# Initialize logging
log_filename = f"tensor_factorization_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

# Initialize results list to store experiment data dynamically
results = []

def run_tf(dim, mode, rank, method, iterations=1000):
    start_time = time.time()

    dim = (dim,)
    shape = dim * mode
    tensor = torch.randn(shape)
    constraint = (torch.rand_like(tensor) > 0.5).float()

    if method == "cp":
        rank = rank
    elif method == "tucker":
        rank = (rank,) * mode
    elif method == "train":
        rank = [1] + [rank] * (mode - 1) + [1]
    elif method == "ring":
        rank = rank

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tf = TensorFactorization(tensor, rank=rank, method=method, mask=None, constraint=constraint, device=device)

    try:
        factors = tf.optimize()
        reconstruction = tf.reconstruct()

        # Extract loss details
        tf.optimize(lr=0.01, max_iter=iterations, tol=1e-6, reg_lambda=0.01, constraint_lambda=1)

        loss = tf.loss
        mse_loss = tf.mse_loss
        constraint_loss = tf.constraint_loss
        l2_loss = tf.l2_loss

        params = tf.total_params

        elapsed_time = time.time() - start_time

        # Log results
        results.append({
            "Mode": mode,
            "Method": method,
            "Params": params,
            "Iter": iterations-1,  # Fixed value based on experiment
            "Loss": loss.item(),
            "MSE": mse_loss.item(),
            "CONST": constraint_loss.item(),
            "L2": l2_loss.item(),
            "Time (s)": elapsed_time,
        })

        logging.info(f"Mode: {mode}, Method: {method}, Loss: {loss.item()}, Time: {elapsed_time:.6f} seconds")
    except Exception as e:
        logging.error(f"Error occurred for mode: {mode}, method: {method}. Exception: {e}")


# Main experiment loop
dim = 7
rank = 3
methods = ["cp", "tucker", "train", "ring"]
modes = [2, 3, 4, 5, 6, 7, 8, 9]

iterations = 10000

for mode in modes:
    for method in methods:
        run_tf(dim=dim, mode=mode, rank=rank, method=method, iterations=iterations)

# Create DataFrame from the results
df = pd.DataFrame(results)

Initialized cp decomposition with rank 3 on device cuda.
Total parameters: 42
Iter: 999, Loss: 1.360875129699707
MSE: 0.3787766098976135, CONST: 0.6882670521736145, L2: 0.29383140802383423
Converged.
Initialized tucker decomposition with rank (3, 3) on device cuda.
Total parameters: 51
Converged.
Iter: 9999, Loss: 0.5013117790222168
MSE: 0.1575111597776413, CONST: 0.005437207408249378, L2: 0.3383634090423584
Initialized train decomposition with rank [1, 3, 1] on device cuda.
Total parameters: 42
Iter: 999, Loss: 1.6952437162399292
MSE: 0.7493141293525696, CONST: 0.42801642417907715, L2: 0.5179132223129272
Converged.
Initialized ring decomposition with rank 3 on device cuda.
Total parameters: 126
Iter: 999, Loss: 3.1712253093719482
MSE: 2.1646480560302734, CONST: 0.1721809208393097, L2: 0.8343963623046875
Converged.
Initialized cp decomposition with rank 3 on device cuda.
Total parameters: 63
Iter: 999, Loss: 102.85558319091797
MSE: 93.52955627441406, CONST: 8.8704195022583, L2: 0.45560

ERROR:root:Error occurred for mode: 4, method: cp. Exception: 'NoneType' object has no attribute 'item'


Converged.
Initialized tucker decomposition with rank (3, 3, 3, 3) on device cuda.
Total parameters: 165
Iter: 999, Loss: 1234.65771484375
MSE: 1213.54150390625, CONST: 20.027664184570312, L2: 1.0884617567062378
Converged.
Initialized train decomposition with rank [1, 3, 3, 3, 1] on device cuda.
Total parameters: 168
Iter: 999, Loss: 1158.944580078125
MSE: 1116.5921630859375, CONST: 41.093387603759766, L2: 1.2589995861053467
Iter: 9999, Loss: 1006.6466064453125
MSE: 987.5298461914062, CONST: 17.017467498779297, L2: 2.099301338195801
Initialized ring decomposition with rank 3 on device cuda.
Total parameters: 252
Iter: 999, Loss: 1166.82177734375
MSE: 1139.3939208984375, CONST: 25.50283432006836, L2: 1.9250134229660034
Converged.
Initialized cp decomposition with rank 3 on device cuda.
Total parameters: 105
Iter: 999, Loss: 8675.291015625
MSE: 8644.6015625, CONST: 30.33057403564453, L2: 0.3587978184223175
Converged.
Initialized tucker decomposition with rank (3, 3, 3, 3, 3) on device cu

ERROR:root:Error occurred for mode: 5, method: train. Exception: 'NoneType' object has no attribute 'item'


Converged.
Initialized ring decomposition with rank 3 on device cuda.
Total parameters: 315
Iter: 999, Loss: 8661.3994140625
MSE: 8526.208984375, CONST: 133.29220581054688, L2: 1.8983792066574097
Converged.
Initialized cp decomposition with rank 3 on device cuda.
Total parameters: 126
Iter: 999, Loss: 58942.59375
MSE: 58890.765625, CONST: 51.361473083496094, L2: 0.4665127098560333
Converged.
Initialized tucker decomposition with rank (3, 3, 3, 3, 3, 3) on device cuda.
Total parameters: 855
Iter: 999, Loss: 61344.046875
MSE: 59754.96484375, CONST: 1581.1092529296875, L2: 7.9721198081970215
Converged.
Initialized train decomposition with rank [1, 3, 3, 3, 3, 3, 1] on device cuda.
Total parameters: 294
Iter: 999, Loss: 62097.1484375
MSE: 60020.06640625, CONST: 2075.1337890625, L2: 1.9489257335662842
Converged.
Initialized ring decomposition with rank 3 on device cuda.
Total parameters: 378
Iter: 999, Loss: 65904.484375
MSE: 63723.390625, CONST: 2178.68017578125, L2: 2.4125723838806152
Con

In [15]:
df

,Mode,Method,Params,Iter,Loss,MSE,CONST,L2,Time (s)
0,2,cp,42,9999,1.360875e+00,3.787766e-01,6.882671e-01,0.293831,23.576854
1,2,tucker,51,9999,5.013118e-01,1.575112e-01,5.437207e-03,0.338363,24.677812
2,2,train,42,9999,1.695244e+00,7.493141e-01,4.280164e-01,0.517913,10.104060
3,2,ring,126,9999,3.171225e+00,2.164648e+00,1.721809e-01,0.834396,7.563452
4,3,cp,63,9999,1.028556e+02,9.352956e+01,8.870420e+00,0.455605,12.879019
5,3,tucker,90,9999,4.784948e+01,4.387678e+01,3.321077e+00,0.651628,29.275353
6,3,train,105,9999,6.409468e+01,5.176029e+01,1.078234e+01,1.552056,27.655308
7,3,ring,189,9999,1.148554e+02,9.937906e+01,1.355647e+01,1.919879,26.848555
8,4,tucker,165,9999,1.234658e+03,1.213542e+03,2.002766e+01,1.088462,6.914303
9,4,train,168,9999,1.006647e+03,9.875298e+02,1.701747e+01,2.099301,32.160262
